In [1]:
import requests
from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException, NoSuchElementException
# import time
import pandas as pd
import datetime as dt

In [2]:
# 기본 설정
hdr = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
    }
categories = ["science", "game", "universe", "it", "ai", "technology"]
start_date = "20240731"
date_obj = dt.datetime.strptime(start_date, "%Y%m%d")

In [3]:
date_obj

datetime.datetime(2024, 7, 31, 0, 0)

In [4]:
# 마지막 페이지 번호 가져오기
def get_last_page_num(category:str, date:str):
    pgn = 1
    first_url = f"https://news.daum.net/breakingnews/digital/{category}?page={pgn}&regDate={date}"
    res = requests.get(first_url, headers=hdr)
    soup = BeautifulSoup(res.text, 'html.parser')

    if soup.select("a.btn_page.btn_next") == []:
        return soup.select("a.num_page")
    else:
        while soup.select("a.btn_page.btn_next") != []:
            pgn += 10
            first_url = f"https://news.daum.net/breakingnews/digital/{category}?page={pgn}&regDate={date}"
            res = requests.get(first_url, headers=hdr)
            soup = BeautifulSoup(res.text, 'html.parser')
        return soup.select("a.num_page")

In [5]:
# 1달 날짜리스트 생성
def make_monthly_day_list(date_obj) -> tuple:
    current_month = date_obj.month
    monthly_day_list = []

    while date_obj.month == current_month:
        monthly_day_list.append((date_obj.strftime("%Y%m%d")))
        
        date_obj -= dt.timedelta(days=1)

    return date_obj, monthly_day_list

In [6]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# caps = DesiredCapabilities().CHROME
# caps["pageLoadStrategy"] = "none"

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36")


def source_from_news(news_url:str):
    
    # driver = webdriver.Chrome(options=chrome_options)  # Or use Firefox(), Edge(), etc.
    # driver.get(news_url)  # Replace with the actual URL

    # html = driver.page_source
    res = requests.get(news_url, headers=hdr)
    soup = BeautifulSoup(res.text, 'html.parser')

    # count_list = soup.select('span.jsx-2157231875.🎬_count_label')
    # recommend = [i.get_text() for i in count_list]
    # recommend

    if soup.select("h3.tit_view") != []:
        title = soup.select("h3.tit_view")[0].get_text()
    else:
        title = None

    if soup.select("h1.doc-title") != []:
        press = soup.select("h1.doc-title")[0].text.strip()
    else:
        press = None

    if soup.select('[dmcf-ptype="general"]') != []:
        reporter = soup.select('[dmcf-ptype="general"]')[-1].get_text()
    else:
        reporter = None

    if soup.select('[dmcf-ptype="general"]') != []:
        content = " ".join([i.get_text() for i in soup.select('[dmcf-ptype="general"]')][:-1])
    else:
        content = None
        
    return title, press, reporter, content
# recommend

In [7]:
# 한달치 뉴스데이터
month_list = []
more_page_list = []

title_list = []
category_list = []
url_list = []
date_list = []
press_list = []
reporter_list = []
content_list = []
recommend_list = []

temp = make_monthly_day_list(date_obj)
month_date_list = temp[1]
date_obj = temp[0]
month_list.append(date_list)


for category in categories: # 카테고리 선택
    
    for date in month_date_list: # date 선택
        
        if get_last_page_num(category, date) == []: #빈페이지일 경우 다음 날짜로
            continue
        else:
            last_page_num = get_last_page_num(category, date)[-1].get_text() # 마지막페이지 가져오기

        for page_num in range(1, int(last_page_num)+1): # 모든 페이지의 모든 기사 url 리스트 가져오기
            page_url = f"https://news.daum.net/breakingnews/digital/{category}?page={page_num}&regDate={date}"
            
            res2 = requests.get(page_url, headers=hdr)
            soup2 = BeautifulSoup(res2.text, 'html.parser')

            newslink_in_page = [i.attrs['href'] for i in soup2.select("ul.list_news2.list_allnews li strong.tit_thumb a.link_txt")]
            
            for newslink in newslink_in_page: # 모든 기사링크에서 source 가져오기
                
                source = source_from_news(newslink)

                title_list.append(source[0])
                category_list.append(category)
                url_list.append(newslink)
                date_list.append(date)
                press_list.append(source[1])
                reporter_list.append(source[2])
                content_list.append(source[3])
                # recommend_list.append(source[4])

                
        print(f"{date} 완료")
    print(f"{category} 완료")

20240731 완료
20240730 완료
20240729 완료
20240728 완료
20240726 완료
20240725 완료
20240724 완료
20240723 완료
20240722 완료
20240721 완료
20240719 완료
20240718 완료
20240717 완료
20240716 완료
20240715 완료
20240714 완료
20240713 완료
20240712 완료
20240711 완료
20240710 완료
20240709 완료
20240708 완료
20240707 완료
20240705 완료
20240704 완료
20240703 완료
20240702 완료
20240701 완료
science 완료
20240731 완료
20240730 완료
20240729 완료
20240728 완료
20240727 완료
20240726 완료
20240725 완료
20240724 완료
20240723 완료
20240722 완료
20240721 완료
20240719 완료
20240718 완료
20240717 완료
20240716 완료
20240715 완료
20240714 완료
20240712 완료
20240711 완료
20240710 완료
20240709 완료
20240708 완료
20240707 완료
20240705 완료
20240704 완료
20240703 완료
20240702 완료
20240701 완료
game 완료
universe 완료
it 완료
ai 완료
technology 완료


In [20]:
title_list

['PAKISTAN INTERNET',
 '“도대체 몇 개야?” 다리에 팔까지 달린 ‘정체’…알고보니',
 '한국로봇산업협회 부회장에 박철완 서정대 교수',
 '수련 특례도 무용… 내년 의사 배출 `절벽`',
 '원자력硏-육군화생방학교, 국가안보 강화 맞손',
 '일동제약, 2분기 영업익 49억원…전년대비 흑자 전환',
 '‘이달의 과기인상’에 김창석 부산대 교수',
 '과학기술 특화 AI 언어모델 나왔다',
 '바이오엔테크 "mRNA 암 백신, 악성 피부암에 효과"',
 '아이큐어, 사옥 610억원에 매각…재무구조 개선 달성',
 "'4세대 원전' 고온가스로, 민·관 함께 만든다",
 '노벨티노빌리티, 코스닥 기술성 평가 통과 “하반기 예비심사 청구”',
 '귀여운 건 못 참는 Z세대 완구社 미니어처로 공략',
 '보령 2분기 영업익 201억원…카나브패밀리, 반기 첫 700억 돌파',
 '마상배 대표 "커피머신 같은 가정용 진단기기 내놓겠다"',
 'HK이노엔, 2분기 영업익 59% 증가…“전문의약품 실적↑"',
 '[과학게시판] 원자력연, 육군화생방학교와 협력 강화 위한 MOU 체결 外',
 '셀트리온·셀트리온제약, 합병 타당성 검토 착수',
 '한국 대표단, 아세안+3 중학생 과학실험 탐구대회 금1·은2·동3',
 '4D 라이다 개발···자율주행 상용화 진전 [이달의 과학기술인상]',
 "우주선 충돌로 궤도 바뀐 소행성, '30만 살' 나이 확인됐다",
 '`사망률 1위 `폐암`…코어라인소프트, AI 의료기술로 조기 발견 돕는다',
 '최수진 의원 "연구개발사업에 융자지원 방식 도입해야"',
 '실적 상승에도 경영권 분쟁에 주주들 한숨…한미약품 모녀, 이사회 재편 예고',
 '[과기원NOW] KAIST, 신개념 제노-프리 줄기세포 배양 플랫폼 개발 外',
 '[사이언스게시판] 원자력연, 육군화생방학교와 국가 안보 강화 나서 外',
 '"英 \'과학 자본\' 개념 부상...이공계 인재부족 겪는 韓도 주목해야"',
 '\'합성고분자 만능줄기세포\' 세계 첫 

In [21]:
len(press_list)

5551